## Бейзлайн - простейший плоский классификатор на основе библиотеки FastText.

In [1]:
import os
from pathlib import Path
import fasttext
import numpy as np 
import pandas as pd
import csv
from gensim.utils import simple_preprocess

from HierarchicalLibrary import CategoryTree, Classifier

In [2]:
# Method for increasing the weight of the first words of title
def word_pyramid(string: str, min_n_words: int, max_n_words: int) -> list:
    result = []
    split = string.split(' ')
    for i in range(min_n_words, max_n_words+1):
        result += split[:i]
    return ' '.join(result)

# Predicting labels and probabilities for list of documents 
def predict_proba(documents: list) -> tuple:
    prediction = model.predict(documents, k=1)
    labels_result = []
    proba_result = []
    for label in prediction[0]:
        labels_result.append(int(label[0][9:]))
    return np.array(labels_result), np.array(prediction[1])[:, 0]

# Predicting on a single input
def predict(document):
    return int(model.predict(document)[0][0][9:])

# Test data preparation
def get_prepared_test_data(df_in: pd.DataFrame) -> pd.DataFrame:
    df=df_in.copy()
    df.drop(['rating', 'feedback_quantity'], axis=1, inplace=True)
    df.title = df.title.astype('string')
    df.short_description = df.short_description.astype('string')
    df.fillna(value='', inplace=True)
    df.name_value_characteristics = df.name_value_characteristics.astype('string')
    df = df.assign(Document=[str(x) + ' ' + str(y) + ' ' + str(z) + ' ' + word_pyramid(x, 2, 3) for x, y, z in zip(df['title'], df['short_description'], df['name_value_characteristics'])])
    df.drop(['title', 'short_description', 'name_value_characteristics'], axis=1, inplace=True)
    df.Document = df.Document.astype('string')
    df.Document = df.Document.apply(lambda x: ' '.join(simple_preprocess(x)))
    return df

def get_fasttext_multilabels(cat_tree: object, category_id: int) -> str:
    node_list = []
    for node in cat_tree.get_id_path(category_id):
        node_list.append(''.join(['__label__', str(node)]))
    return ' '.join(node_list)
    

Подготавливаем полный, тренировочный и валидационный датасеты:
перемешиваем данные в фрейме,
удаляем колонки рейтинга и кол-ва отзывов,
корректируем типы данных колонок,
заполняем пропущенные значения,
текст из колонок 'title', 'short_description' и 'name_value_characteristics' объединяем в колонку "Document", добавляем первые слова из колонки 'title', чтобы увеличить их вес (самые важные слова - в начале описания).

In [3]:
full_train_data = pd.read_parquet('train.parquet')

data_full = full_train_data.sample(frac=1, random_state=1).copy()
data_full.drop(['rating', 'feedback_quantity'], axis=1, inplace=True)
data_full.title = data_full.title.astype('string')
data_full.short_description = data_full.short_description.astype('string')
data_full.fillna(value='', inplace=True)
data_full.name_value_characteristics = data_full.name_value_characteristics.astype('string')
data_full = data_full.assign(Document=[str(x) + ' ' + str(y) + ' ' + str(z) + ' ' + word_pyramid(x, 2, 3) for x, y, z in zip(data_full['title'], data_full['short_description'], data_full['name_value_characteristics'])])
data_full.drop(['title', 'short_description', 'name_value_characteristics'], axis=1, inplace=True)
data_full.Document = data_full.Document.astype('string')

data = data_full[:-4000].reset_index(drop=True)
data_valid = data_full[-4000:].reset_index(drop=True)


Строим дерево каталога

In [4]:
cat_tree_df = pd.read_csv('categories_tree.csv', index_col=0)

In [5]:
cat_tree = CategoryTree()
cat_tree.add_nodes_from_df(cat_tree_df, parent_id_col='parent_id', title_col='title')
cat_tree.add_goods_from_df(data_full, category_id_col='category_id', good_id_col='id')

In [6]:
classifier = Classifier()

Преобразуем данные в формат, принимаемый FastText.

In [7]:
data.Document = data.Document.apply(lambda x: ' '.join(simple_preprocess(x)))
data_valid.Document = data_valid.Document.apply(lambda x: ' '.join(simple_preprocess(x)))

data.category_id = data.category_id.apply(lambda x: get_fasttext_multilabels(cat_tree, x))
data_valid.category_id = data_valid.category_id.apply(lambda x: get_fasttext_multilabels(cat_tree, x))

In [8]:
data.iloc[1]

id                                                        304936
category_id    __label__12917 __label__11328 __label__10091 _...
Document       силиконовый дорожный контейнер футляр чехол дл...
Name: 1, dtype: object

FastText принимает данные в виде текстовых файлов, поэтому сохраняем данные на диск.

In [9]:
# Saving the CSV file as a text file to train/test the classifier
data[['Document', 'category_id']].to_csv('train_fasttext.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

data_valid[['Document', 'category_id']].to_csv('test_fasttext.txt', 
                                               index = False, 
                                               sep = ' ',
                                               header = None, 
                                               quoting = csv.QUOTE_NONE, 
                                               quotechar = "", 
                                               escapechar = " ")


Обучаем модель.

In [18]:
# Training the fastText classifier
model = fasttext.train_supervised('train_fasttext.txt',
                                  lr=0.3,                
                                  dim=45,               
                                  ws=5, 
                                  loss='ova',                                  
                                  epoch=30,             
                                  neg=500,               
                                  wordNgrams=3)
classifier.fasttext = model

Read 5M words
Number of words:  87455
Number of labels: 1475
Progress: 100.0% words/sec/thread:   46260 lr:  0.000000 avg.loss:  1.083445 ETA:   0h 0m 0s 11.7% words/sec/thread:   62115 lr:  0.264964 avg.loss:  4.354555 ETA:   0h 8m11s 12.8% words/sec/thread:   62236 lr:  0.261665 avg.loss:  4.147582 ETA:   0h 8m 4s 3.530655 ETA:   0h 7m39s words/sec/thread:   53332 lr:  0.174868 avg.loss:  2.027093 ETA:   0h 6m17s ETA:   0h 2m 3s 96.2% words/sec/thread:   46170 lr:  0.011430 avg.loss:  1.118221 ETA:   0h 0m28s


При необходимости, сохраняем или загружаем модель.

In [19]:
# Save the trained model
path = os.path.join(Path(".").parent, 'FastText_baseline', )
model.save_model('fasttext_multilabel_model')

In [20]:
# Load the trained model
classifier.fasttext = fasttext.load_model('fasttext_multilabel_model')

Проверяем качество классификации на трейне:

In [21]:
# Evaluating performance on the entire train file
_, precision, recall = classifier.fasttext.test('train_fasttext.txt') 
leaf_F1 = (2*precision*recall) / (precision+recall)

In [22]:
print(f'Train Leaf_F1={leaf_F1:.4f}') 

Train Leaf_F1=0.4094


Проверяем качество классификации на тестовой выборке:

In [23]:
# Evaluating performance on the entire test file
_, precision, recall = classifier.fasttext.test('test_fasttext.txt')                      
leaf_F1 = (2*precision*recall) / (precision+recall)

In [24]:
print(f'Test Leaf_F1={leaf_F1:.4f}') 

Test Leaf_F1=0.4000
